# **CPC353 Assignment 2**

## **This assignment is to classifying English-Malay sentences according to their semantics**

I used the GloVe 6B 100d pre-trained dwonloaded from [kaggle](https://www.kaggle.com/datasets/danielwillgeorge/glove6b100dtxt?resource=download) as the pre-trained embedding to embed the word vocabularies from the datasets given:


1.   `train.en` - training files of English Sentences
2.   `train.my` - training files of Malay Sentences
3. `train.cl` - training files of the classes
4. `test.en` - testing files of English Sentences
5. `test.my` - testing files of Malay Sentences
6. `test.cl` - testing files of the classes
7. `dev.en` - validating files of English Sentences
8. `dev.my` - validating files of Malay Sentences
9. `dev.cl` - validating files of the classes

There are 3 files for each training, testing and validation sets. The .en files are english sentences, the .my files are malay sentences, and the .cl files are the labels for english and malay files to indicate whether are both sentences similar. The .cl files are just 1s and 0s.

The notebook is sectioned into multiple steps: Tokenizing text, embedding text, building neural network, and testing the network.

### Neural network architecture
The neural network architecture starts with two models. One for the English language, one for the Malay language. After that, I will concatenate both models into one layer where it will perform binary classification.

### **Step 1: Reading the Text Files and Tokenizing the Texts**

I will first read the files into a list of sentences.

In [ ]:
def read_text(text_path):
    # This function will read the txt file and split the sentences with new line character, and will also remove the
    # last character since it is an empty string
    return open(text_path, "r", encoding="utf8").read().split("\n")[:-1]

train_text_en = read_text("/kaggle/input/english-malay/train.en")
train_text_my = read_text("/kaggle/input/english-malay/train.my")
train_label = read_text("/kaggle/input/english-malay/train.cl")

test_text_en = read_text("/kaggle/input/english-malay/test.en")
test_text_my = read_text("/kaggle/input/english-malay/test.my")
test_label = read_text("/kaggle/input/english-malay/test.cl")

dev_text_en = read_text("/kaggle/input/english-malay/dev.en")
dev_text_my = read_text("/kaggle/input/english-malay/dev.my")
dev_label = read_text("/kaggle/input/english-malay/dev.cl")

In [ ]:
# Defining all imports for the project
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Embedding, Input, Concatenate
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras import Model

Tokenization of text will be carried using the Tokenizer class from keras. Since I will be concatenating two models for two langauges. That means that I will have to tokenize both english and malay sentences separately.

In [ ]:
# Tokenize the text data
en_tokenizer = Tokenizer()
en_tokenizer.fit_on_texts(train_text_en)
en_sequences = en_tokenizer.texts_to_sequences(train_text_en)

my_tokenizer = Tokenizer()
my_tokenizer.fit_on_texts(train_text_my)
my_sequences = my_tokenizer.texts_to_sequences(train_text_my)

# Find the maximum length between the two sequences, so we can pad them
max_length = max(len(max(en_sequences, key=len)), len(max(my_sequences, key=len)))

# Padding the sequences to same length
en_sequences = pad_sequences(en_sequences, maxlen=max_length)
my_sequences = pad_sequences(my_sequences, maxlen=max_length)

In [ ]:
# Showing the number of vocabulary in both languages training data
# Uncomment for the tokens and sequences

print("Number of unique words in English text data: ", len(en_tokenizer.word_index))
# print("English text data is: ", en_tokenizer.word_index)
# print("English seqeunce: ", en_sequences)

print("Number of unique words in Malay text data: ", len(my_tokenizer.word_index))
# print("Malay text data is: ", my_tokenizer.word_index)
# print("Malay seqeunce: ", my_sequences)

Number of unique words in English text data:  39908
Number of unique words in Malay text data:  23279
